# POST an EAMENA dataset on Zenodo

Post an EAMENA dataset on Zenodo from an EAMENA GeoJSON URL using the Zenodo API. See also the documentation in GitHub [README](https://github.com/eamena-project/eamena-arches-dev/blob/main/data/bibref/README.md) file

---

Zenodo documentation: https://developers.zenodo.org/#quickstart-upload

## Functions and Libraries

In [1]:
!rm eamena-functions -R
!git clone https://github.com/eamena-project/eamena-functions.git
%cd /content/eamena-functions/zenodo
import zenodo as zn

import os
import requests
import json
import zipfile
import pandas as pd
import numpy as np
!pip install sickle
import sickle

# needed to export as JSON
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

Cloning into 'eamena-functions'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 250 (delta 15), reused 32 (delta 10), pack-reused 210
Receiving objects: 100% (250/250), 56.59 KiB | 3.33 MiB/s, done.
Resolving deltas: 100% (138/138), done.
/content/eamena-functions/zenodo


### Query EAMENA DB

Query the database API using the `GEOJSON_URL` URL selection. Creates a GeoJSON data.

In [2]:
## GeoJSON URL
# HPs
# GEOJSON_URL = "https://database.eamena.org/api/search/export_results?paging-filter=1&tiles=true&format=geojson&reportlink=false&precision=6&language=*&total=1641&resource-type-filter=%5B%7B%22graphid%22%3A%2234cfe98e-c2c0-11ea-9026-02e7594ce0a0%22%2C%22name%22%3A%22Heritage%20Place%22%2C%22inverted%22%3Afalse%7D%5D&map-filter=%7B%22type%22%3A%22FeatureCollection%22%2C%22features%22%3A%5B%7B%22type%22%3A%22Feature%22%2C%22properties%22%3A%7B%22Grid%20ID%22%3A%22E60N30-24%22%2C%22buffer%22%3A%7B%22width%22%3A%220%22%2C%22unit%22%3A%22m%22%7D%2C%22inverted%22%3Afalse%7D%2C%22geometry%22%3A%7B%22type%22%3A%22Polygon%22%2C%22coordinates%22%3A%5B%5B%5B60.5%2C31.25%5D%2C%5B60.5%2C31.5%5D%2C%5B60.75%2C31.5%5D%2C%5B61%2C31.5%5D%2C%5B61.25%2C31.5%5D%2C%5B61.5%2C31.5%5D%2C%5B61.75%2C31.5%5D%2C%5B62%2C31.5%5D%2C%5B62%2C31.25%5D%2C%5B62.25%2C31.25%5D%2C%5B62.25%2C31%5D%2C%5B62.25%2C30.75%5D%2C%5B62%2C30.75%5D%2C%5B62%2C30.5%5D%2C%5B61.75%2C30.5%5D%2C%5B61.5%2C30.5%5D%2C%5B61.5%2C30.25%5D%2C%5B61.25%2C30.25%5D%2C%5B61%2C30.25%5D%2C%5B60.75%2C30.25%5D%2C%5B60.75%2C30.5%5D%2C%5B60.75%2C30.75%5D%2C%5B61%2C30.75%5D%2C%5B61%2C31%5D%2C%5B60.75%2C31%5D%2C%5B60.75%2C31.25%5D%2C%5B60.5%2C31.25%5D%5D%5D%7D%7D%5D%7D"
# GS
GEOJSON_URL = "https://database.eamena.org/api/search/export_results?paging-filter=1&tiles=true&format=geojson&reportlink=false&precision=6&language=*&total=25&resource-type-filter=%5B%7B%22graphid%22%3A%2277d18973-7428-11ea-b4d0-02e7594ce0a0%22%2C%22name%22%3A%22Grid%20Square%22%2C%22inverted%22%3Afalse%7D%5D&map-filter=%7B%22type%22%3A%22FeatureCollection%22%2C%22features%22%3A%5B%7B%22type%22%3A%22Feature%22%2C%22properties%22%3A%7B%22id%22%3A1%2C%22buffer%22%3A%7B%22width%22%3A%220%22%2C%22unit%22%3A%22m%22%7D%2C%22inverted%22%3Afalse%7D%2C%22geometry%22%3A%7B%22type%22%3A%22MultiPolygon%22%2C%22coordinates%22%3A%5B%5B%5B%5B61.9185199386503%2C31.434624233128837%5D%2C%5B61.93140337423312%2C31.176955521472394%5D%2C%5B62.17296779141103%2C31.164072085889572%5D%2C%5B62.195513803680974%2C30.829102760736195%5D%2C%5B61.91207822085889%2C30.829102760736195%5D%2C%5B61.92496165644171%2C30.587538343558283%5D%2C%5B61.396740797546%2C30.593980061349694%5D%2C%5B61.40318251533741%2C30.316986196319018%5D%2C%5B60.797661042944775%2C30.320207055214723%5D%2C%5B60.810544478527596%2C30.68094325153374%5D%2C%5B61.07465490797545%2C30.68094325153374%5D%2C%5B61.06821319018404%2C31.083550613496932%5D%2C%5B60.82342791411042%2C31.089992331288343%5D%2C%5B60.80732361963189%2C31.312231595092026%5D%2C%5B60.54643404907974%2C31.318673312883437%5D%2C%5B60.53355061349692%2C31.437845092024542%5D%2C%5B61.9185199386503%2C31.434624233128837%5D%5D%5D%5D%7D%7D%5D%7D"
resp = requests.get(GEOJSON_URL)
data = resp.json()

### Generate metadata from the GeoJSON data

Except for *free text* fields, metadata is collected from the GeoJSON file (see [eamena-functions](https://github.com/eamena-project/eamena-functions/blob/main/zenodo/zenodo.py))

---

see the [list of Zenodo metadata](https://developers.zenodo.org/#depositions) list (under 'Deposit metadata' chapter)

#### Free text

In [3]:
## TITLE, DESCRIPTION and the GeoJSON filename
# HPs
# TITLE = "Sistan: part 1. Heritage Places" # free text
# DESCRIPTION = "Deposit of Heritage Places from the Sistan region (Iran, Afghanistan) maintained within the EAMENA database" # free tex
# FILENAME = "sistan_part1_hps"
# ADDITIONAL_KEYWORDS = ['Climate Change']
# GS
TITLE = "Sistan: part 1. Grid Squares" # free text
DESCRIPTION = "Deposit of Grid Squares from the Sistan region (Iran, Afghanistan) maintained within the EAMENA database" # free tex
FILENAME = "sistan_part1_gs"
ADDITIONAL_KEYWORDS = []

#### Calculated metadata

In [4]:
metadata = {
     'metadata': {
         'title': TITLE,
         'description': DESCRIPTION,
         'upload_type': 'dataset',
         'license': 'cc-by',
         'subjects': [{"term": "Cultural property", "identifier": "https://id.loc.gov/authorities/subjects/sh97000183.html", "scheme": "url"}],
         'method': 'EAMENA data entry methodology',
         'creators': [{'name': "EAMENA database",
                       'affiliation': "University of Oxford, University of Southampton"}],
         'contributors': zn.zenodo_contributors(data),
         'keywords': zn.zenodo_keywords(data, additional=ADDITIONAL_KEYWORDS),
         'dates': zn.zenodo_dates(data),
        #  'communities': "[{'identifier': 'eamena'}]",
        #  'related_identifiers': zn.zenodo_related_identifiers()
     }
 }

print(json.dumps(metadata, indent=4))

{
    "metadata": {
        "title": "Sistan: part 1. Grid Squares",
        "description": "Deposit of Grid Squares from the Sistan region (Iran, Afghanistan) maintained within the EAMENA database",
        "upload_type": "dataset",
        "license": "cc-by",
        "subjects": [
            {
                "term": "Cultural property",
                "identifier": "https://id.loc.gov/authorities/subjects/sh97000183.html",
                "scheme": "url"
            }
        ],
        "method": "EAMENA data entry methodology",
        "creators": [
            {
                "name": "EAMENA database",
                "affiliation": "University of Oxford, University of Southampton"
            }
        ],
        "contributors": [
            {
                "name": "University of Oxford",
                "type": "DataManager"
            },
            {
                "name": "University of Southampton",
                "type": "DataManager"
            }
        ],
    

ℹ️ creates an issue when uploading the grants: `'grants': [{'id': '051z6e826::4178'}]` confirmed by the Zenodo support

ℹ️  not working when uploading on https://zenodo.org/api/deposit/depositions or https://sandbox.zenodo.org/. Has to be completed in the GUI:
* community : `'communities': "[{'identifier': 'eamena'}]"`
* related identifiers `'related_identifiers': zn.zenodo_related_identifiers()`

### Data output

Write JSON and ZIP locally

In [5]:
json_file_name = FILENAME + ".geojson"
zip_file_name = FILENAME + ".zip"
# Create the JSON file and write the data to it
json_string = json.dumps(data, cls = NpEncoder)
json_string = json.loads(json_string)
with open(json_file_name, 'w') as json_file:
	json.dump(json_string, json_file, indent=4)
	print(json_file_name + " has been exported in " + os.getcwd())

# Create a ZIP file and add the JSON file to it
with zipfile.ZipFile(zip_file_name, "w", zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(json_file_name)
    print(zip_file_name + " has been exported in " + os.getcwd())

sistan_part1_gs.geojson has been exported in /content/eamena-functions/zenodo
sistan_part1_gs.zip has been exported in /content/eamena-functions/zenodo


ℹ️ Further data can be created and files added into the ZIP

## Zenodo

### Creates an empty bucket

Paste your `ACCESS_TOKEN` (see Zenodo documentation [here](https://zenodo.org/account/settings/applications/tokens))

In [6]:
ACCESS_TOKEN = ''

Choose one of selected Zenodo deposit:
* Zenodo (`ZENODO_URL = 'https://zenodo.org/api/deposit/depositions'`)`
* Zenodo sanbox for tests (`ZENODO_URL = 'https://sandbox.zenodo.org/api/deposit/depositions'`)

In [7]:
ZENODO_URL = 'https://zenodo.org/api/deposit/depositions'

Create the bucket

In [8]:
params = {'access_token': ACCESS_TOKEN}
r = requests.post(ZENODO_URL,
                   params=params,
                   json={})
r.status_code
r.json()
# collect the deposition id
deposition_id = r.json()['id']
print("The deposition_id is: " + str(deposition_id))

The deposition_id is: 10376132


### Add data

In [9]:
bucket_url = r.json()["links"]["bucket"]
with open(zip_file_name, "rb") as fp:
    r = requests.put(
        "%s/%s" % (bucket_url, zip_file_name),
        data = fp,
        params = params,
    )
r.json()

{'created': '2023-12-14T10:21:10.957819+00:00',
 'updated': '2023-12-14T10:21:11.005229+00:00',
 'version_id': '0710dc9f-57c4-4660-9220-01199f7f70a1',
 'key': 'sistan_part1_gs.zip',
 'size': 817,
 'mimetype': 'application/zip',
 'checksum': 'md5:ae1dc017c8771164c52ad7dacac6391d',
 'is_head': True,
 'delete_marker': False,
 'links': {'self': 'https://zenodo.org/api/files/0dc7d979-60c3-40d9-8c19-e43a6857d208/sistan_part1_gs.zip',
  'version': 'https://zenodo.org/api/files/0dc7d979-60c3-40d9-8c19-e43a6857d208/sistan_part1_gs.zip?versionId=0710dc9f-57c4-4660-9220-01199f7f70a1',
  'uploads': 'https://zenodo.org/api/files/0dc7d979-60c3-40d9-8c19-e43a6857d208/sistan_part1_gs.zip?uploads'}}

### Add metadata

In [10]:
deposition_id

10376132

In [11]:
r = requests.put('%s/%s' % (ZENODO_URL, deposition_id),
                  params = {'access_token': ACCESS_TOKEN},
                  data = json.dumps(metadata)) # ,
                  # headers = headers)
r.status_code
# 200

200

### Publish

In [12]:
r = requests.post('%s/%s/actions/publish' % (ZENODO_URL, deposition_id),
                      params={'access_token': ACCESS_TOKEN} )
r.status_code
# 504

202

### Check

Have a look at the last deposit (`r.json()[0]`)

In [13]:
r = requests.get(ZENODO_URL,
                  params={'access_token': ACCESS_TOKEN})
r.status_code
# 200
r.json()[0]

{'created': '2023-12-14T10:21:31.823898+00:00',
 'modified': '2023-12-14T10:21:31.971888+00:00',
 'id': 10376132,
 'conceptrecid': '10376131',
 'doi': '10.5281/zenodo.10376132',
 'conceptdoi': '10.5281/zenodo.10376131',
 'doi_url': 'https://doi.org/10.5281/zenodo.10376132',
 'metadata': {'title': 'Sistan: part 1. Grid Squares',
  'doi': '10.5281/zenodo.10376132',
  'publication_date': '2023-12-14',
  'description': 'Deposit of Grid Squares from the Sistan region (Iran, Afghanistan) maintained within the EAMENA database',
  'access_right': 'open',
  'creators': [{'name': 'EAMENA database',
    'affiliation': 'University of Oxford, University of Southampton'}],
  'contributors': [{'name': 'University of Oxford',
    'affiliation': None,
    'type': 'DataManager'},
   {'name': 'University of Southampton',
    'affiliation': None,
    'type': 'DataManager'}],
  'keywords': ['EAMENA', 'MaREA', 'Cultural Heritage'],
  'subjects': [{'term': 'Cultural property',
    'identifier': 'https://id.l

#### Latest record

The same information can be retrieved in `r.json()[0]['links']['latest_draft']`:

In [14]:
from IPython.display import display, HTML, Markdown
html_link = r.json()[0]['links']['html']
url_markdown = "[See the latest record]({})".format(html_link)
display(Markdown(url_markdown))

[See the latest record](https://zenodo.org/records/10376132)

The Zenodo link to the record is recorded in `r.json()[0]['links']['html']`

In [15]:
html_link

'https://zenodo.org/records/10376132'

#### Badge

In [16]:
# show badge
from IPython.display import Markdown as md
badge = r.json()[0]['links']['badge']
Markdown("![]({})".format(badge))

![](https://zenodo.org/badge/doi/10.5281%2Fzenodo.10376132.svg)

In [17]:
badge

'https://zenodo.org/badge/doi/10.5281%2Fzenodo.10376132.svg'

#### DOI

Its DOI is `r.json()[0]['doi_url']`

In [18]:
r.json()[0]['doi_url']

'https://doi.org/10.5281/zenodo.10376132'